In [80]:
import pandas as pd
import polars as pl

In [81]:
finances = pl.from_pandas(pd.read_excel(r"C:\Users\paulm\OneDrive\Documents\Finances.xlsx", dtype=str, sheet_name="Monthly", header=None))
dtm_format = "%Y-%m-%d %H:%M:%S"
days_left = finances.filter(pl.col('0').str.starts_with('Days'))['2'].cast(pl.Int32).to_list()[0]
min_salary = finances.filter(pl.col('4').str.starts_with('salary'))['5'].cast(pl.Float64).to_list()[0]

In [82]:
bal = finances.select('0','2')[:4]\
    .rename({'0': 'account', '2': 'balance'})\
    .with_columns(
        pl.col('balance').cast(pl.Float64),
        pl.lit(pd.Timestamp.today().strftime(dtm_format)).alias('udt_dtm')
        )

if days_left >= 31:
    bal = bal.with_columns(
        pl.when(pl.col('account').str.contains("HSBC"))
        .then(pl.col("balance")-min_salary)
        .otherwise(pl.col("balance"))
        .alias("balance")
    )

In [83]:
cards = finances.filter(pl.col('1').str.contains('CC')).select('1','2').rename({'1': 'credit_card', '2': 'outstanding'})\
    .with_columns(
        pl.col('outstanding').cast(pl.Float64),
        pl.lit(pd.Timestamp.today().strftime(dtm_format)).alias('udt_dtm')
        )

In [84]:
exp = finances.filter(
    (pl.col('0').cast(pl.Int32, strict=False).is_not_null())
    & (~pl.col('1').str.contains('CC'))
    )\
    .select('1', '2')\
    .rename({'1': 'expenditure', '2': 'amount'})\
    .with_columns(
        pl.col('amount').cast(pl.Float64),
        pl.lit(pd.Timestamp.today().strftime(dtm_format)).alias('udt_dtm')
        )

In [85]:
sav = finances.filter(pl.col('4').str.to_lowercase()=='expected savings').select('5').rename({'5':'expected_savings'})\
.with_columns(
        pl.col('expected_savings').cast(pl.Float64),
        pl.lit(pd.Timestamp.today().strftime(dtm_format)).alias('udt_dtm')
        )

In [129]:
def stack_hist(df, key):
    hist = pl.read_parquet(f"datacache/{key}.parquet")
    for col in hist.columns:
        hist = hist.with_columns(pl.col(col).cast(df[col].dtype))
    df = hist.vstack(df)
    df.write_parquet(f"datacache/{key}.parquet")
    return df.unique()

In [130]:
hist_dict = {
    "monthly_balance": bal,
    "expenditure": exp,
    "card_spends": cards,
    "savings": sav
}

In [131]:
hist_dict = {
    k: stack_hist(hist_dict[k], k) for k in hist_dict.keys()
}